this notebook is used to calculated the standard deviation of the convertion rate to select the delta used in the UCB Active

In [1]:
import numpy as np
import matplotlib.pyplot as plt


import Code.utils as utils
import Code.environment.settings as settings
from Code.environment.Environment import Environment
from Code.GreedyLearner import GreedyLearner
from Code.environment.Customer import Customer
from Code.UCBLearner import UCBLearner
from Code.TSLearner import TSLearner

%matplotlib inline

In [2]:
cust_json = "customer_classes.json"
today_customer = 100
var_cust = 0
p_lambda = 1.0
graph_json = "business_full_graph.json"

arms = settings.prices
environment = Environment(cust_json,today_customer,var_cust,p_lambda, graph_json, arms)
clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm= environment.estimate_clairvoyant()
print("\n", clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm.values())

loading: [==============================] 99.6%
 [0, 2, 0, 2, 0] 970.7239928 9.707239928 dict_values([752.6665272500001, 693.03838697, 664.4090954200001, 612.0842320700001, 755.492893625, 696.438626435, 668.08338141, 616.290364985, 766.677089, 707.8286694800001, 679.56990328, 628.00226588, 764.833268125, 706.2988147750001, 678.1905866500001, 626.902387525, 734.2235987500001, 674.9221025500002, 646.4497713000002, 594.40910305, 736.8712118750001, 678.1281130250001, 649.92291615, 598.3906682749999, 747.3171700000001, 688.7953492, 660.6947592, 609.3877612, 745.6002593750001, 687.3769441249999, 659.41957475, 608.3764403750001, 725.614559125, 666.3702632850001, 637.92176091, 586.000347635, 728.0888933125001, 669.3930736175001, 641.207364305, 589.7776442925, 737.9615665000001, 679.4994404400001, 651.4248554400002, 600.2207098400002, 736.3202315625001, 678.1458583875001, 650.2094273250001, 599.2578857625001, 678.2049264999999, 620.41278822, 592.6855933200002, 541.58091362, 680.7972857499999, 6

In [2]:
n_arms = 4
n_products = 5 

feature1 = 0
feature2 = 0

customer = Customer(feature1, feature2)

TIME_HORIZON = 300
NUM_ITERATION = 2

In [4]:
aggregate_customer = Customer(
    0,
    0,
    environment.get_aggregate_alphas(),
    environment.get_aggregate_num_prods_distribution(),
    environment.get_aggregate_click_graph()
    )


In [5]:
rewards_iterations = []
conv_rate_hist = []
for i in range(n_products):
    temp = []
    for j in range(n_arms):
        temp.append([])
    conv_rate_hist.append(temp)
        
for i in range(NUM_ITERATION):
    ucb1_learner = UCBLearner(n_arms, n_products, environment.customers, graph_json, arms, environment.customers_distribution)
    print('Iteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ucb1_learner.select_superarm()
        report = environment.round(super_arm)
        ucb1_learner.update(super_arm.copy(), report)
        for p,a in enumerate(super_arm):
            conv_rate_hist[p][a].append(report.get_conversion_rate()[p])
    rewards = ucb1_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

Iteration # 1
loading: [============================= ] 96.7%Iteration # 2
loading: [============================= ] 96.7%

(2, 300)

In [ ]:
rewards_iterations = []
conv_rate_hist = []
for i in range(NUM_ITERATION):
    ts_learner = TSLearner(n_arms, n_products, environment.customers, graph_json, arms, environment.customers_distribution)
    print('\nIteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ts_learner.select_superarm()
        report = environment.round(super_arm)
        ts_learner.update(super_arm.copy(), report)
        conv_rate_hist.append(report.get_conversion_rate())
    rewards = ts_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

In [6]:
conv_rate_hist

[[[0.7368421052631579,
   0.8717948717948718,
   0.8125,
   0.723404255319149,
   0.7924528301886793,
   0.6904761904761905,
   0.7906976744186046,
   0.75,
   0.7291666666666666,
   0.7755102040816326,
   0.825,
   0.8461538461538461,
   0.6842105263157895,
   0.75,
   0.8333333333333334,
   0.7222222222222222,
   0.7560975609756098,
   0.6129032258064516,
   0.8048780487804879,
   0.7647058823529411,
   0.8409090909090909,
   0.7105263157894737,
   0.6744186046511628,
   0.6829268292682927,
   0.7419354838709677,
   0.7555555555555555,
   0.6818181818181818,
   0.7142857142857143,
   0.8095238095238095,
   0.8444444444444444,
   0.66,
   0.7647058823529411,
   0.8260869565217391,
   0.6727272727272727,
   0.723404255319149,
   0.7906976744186046,
   0.75,
   0.6666666666666666,
   0.7777777777777778,
   0.8913043478260869,
   0.7380952380952381,
   0.7659574468085106,
   0.72,
   0.8723404255319149,
   0.7142857142857143,
   0.7111111111111111,
   0.7924528301886793,
   0.79591836734

In [7]:
np.shape(conv_rate_hist)

/Users/sofia/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


(5, 4)

In [8]:
b = []
for i in range(n_products):
    t = []
    for j in range(n_arms):
        t.append([])
    b.append(t)
for p in range(n_products):
    for a in range(n_arms):
        b[p][a] = np.std(conv_rate_hist[p][a])
        
print(b)
        

[[0.05974829251226928, 0.06045064964594729, 0.06781550909916871, 0.05526186115594564], [0.08325006849470215, 0.08395120446002126, 0.07175048421662054, 0.0549699435850392], [0.10866557725209015, 0.12001120950972542, 0.09416056888353792, 0.09322875849284282], [0.12341564079129297, 0.14213753664071105, 0.14924862090059163, 0.14560699991374773], [0.09767738929567515, 0.09896653095464701, 0.08887011551767972, 0.09298055063352471]]


In [4]:
from Code.environment.EnvironmentNonStationary import EnvironmentNonStationary

cust_json = "customer_n_s.json"
today_customer = 100
var_cust = 0
p_lambda = 1.0
graph_json = "business_full_graph.json"
abrupt_change_interval = 100

arms = settings.prices
env = EnvironmentNonStationary(cust_json,today_customer,var_cust,p_lambda, graph_json, arms, abrupt_change_interval)
customers_ns = env.customers_ns.copy()

w1 = []
for i in range(n_products):
    q1 = []
    for j in range(n_arms):
        q1.append([])
    w1.append(q1)
w2 = []
for i in range(n_products):
    q2 = []
    for j in range(n_arms):
        q2.append([])
    w2.append(q2)
w3 = []
for i in range(n_products):
    q3 = []
    for j in range(n_arms):
        q3.append([])
    w3.append(q3)
    
for s in range(n_products):
    for r in range(n_arms):
        for c in customers_ns:
            w1[s][r].append(c.get_buy_distribution()[0][s][r])
            w2[s][r].append(c.get_buy_distribution()[1][s][r])
            w3[s][r].append(c.get_buy_distribution()[2][s][r])
            
print(w1)
print("--------------------")
print(w2)
print("--------------------")
print(w3)
            


[[[0.5, 0.8, 0.8, 0.9], [0.2, 0.5, 0.2, 0.5], [0.1, 0.4, 0.2, 0.3], [0.0, 0.3, 0.2, 0.1]], [[0.7, 0.7, 0.4, 0.4], [0.6, 0.6, 0.8, 0.3], [0.4, 0.35, 0.8, 0.2], [0.1, 0.2, 0.2, 0.1]], [[0.7, 0.9, 0.6, 0.6], [0.5, 0.5, 0.6, 0.5], [0.35, 0.3, 0.4, 0.3], [0.2, 0.2, 0.3, 0.2]], [[0.9, 0.7, 0.3, 0.65], [0.7, 0.5, 0.9, 0.5], [0.6, 0.4, 0.6, 0.4], [0.5, 0.35, 0.3, 0.35]], [[0.5, 0.6, 0.3, 0.7], [0.3, 0.45, 0.4, 0.6], [0.2, 0.25, 0.6, 0.4], [0.1, 0.1, 0.6, 0.3]]]
--------------------
[[[0.3, 0.6, 0.3, 0.7], [0.8, 0.6, 0.7, 0.1], [0.5, 0.7, 0.7, 0.3], [0.4, 0.6, 0.8, 0.9]], [[0.5, 0.3, 0.7, 0.2], [0.9, 0.3, 0.1, 0.8], [0.6, 0.6, 0.8, 0.8], [0.5, 0.1, 0.1, 0.2]], [[0.2, 0.2, 0.6, 0.2], [0.7, 0.9, 0.9, 0.6], [0.1, 0.5, 0.2, 0.3], [0.4, 0.6, 0.2, 0.8]], [[0.1, 0.2, 0.7, 0.2], [0.5, 0.5, 0.5, 0.6], [0.8, 0.2, 0.3, 0.1], [0.4, 0.9, 0.5, 0.1]], [[0.9, 0.8, 0.1, 0.9], [0.1, 0.6, 0.3, 0.1], [0.6, 0.5, 0.2, 0.6], [0.6, 0.1, 0.8, 0.2]]]
--------------------
[[[0.2, 0.9, 0.7, 0.6], [0.4, 0.1, 0.7, 0.6], [0.

In [ ]:
wd1 = []
for i in range(n_products):
    q1 = []
    for j in range(n_arms):
        q1.append([])
    wd1.append(q1)
wd2 = []
for i in range(n_products):
    q2 = []
    for j in range(n_arms):
        q2.append([])
    wd2.append(q2)
wd3 = []
for i in range(n_products):
    q3 = []
    for j in range(n_arms):
        q3.append([])
    wd3.append(q3)
dev = []
for i in range(n_products):
    q4 = []
    for j in range(n_arms):
        q4.append([])
    dev.append(q4)
    

    
